In [ ]:
def bilyeh(data):

    股本 = data.get('股本合計', 1)
    price= data.get('收盤價', 300)
    high = data.get('最高價', 300)
    low  = data.get('最低價', 300)
    成交股數 = data.get('成交股數',1)
    stock_list = price.columns
    d_k={}
    d_d={}
    w_k={}
    w_d={}
    ma_20={}
    for stock in stock_list:
        try:
            pds1 = pd.DataFrame(price[stock])
            pds1.rename(columns={stock:'close'}, inplace=True)
            pds2 = pd.DataFrame(high[stock])
            pds2.rename(columns={stock:'high'}, inplace=True)
            pds3 = pd.DataFrame(low[stock])
            pds3.rename(columns={stock:'low'}, inplace=True)
            pds4 = pd.concat([pds1,pds2,pds3],axis=1)
            tmp1 = abstract.STOCH(pds4,fastk_period=9,slowk_period=3,slowd_period=3)
            d_k.update({stock:tmp1['slowk']})
            d_d.update({stock:tmp1['slowd']})
            tmp2 = abstract.MA(pds4, timeperiod=20, matype=0)
            ma_20.update({stock:tmp2})
        except:
            m=stock
        week = pd.Series(pds4.index.week, pds4.index)
        week = (week.diff() != 0).cumsum()
        w_pds = pds4.groupby(week).apply(lambda s: s.iloc[-1])
        try:
            tmp3 = abstract.STOCH(w_pds,fastk_period=9,slowk_period=3,slowd_period=3)
            w_k.update({stock:tmp3['slowk']})
            w_d.update({stock:tmp3['slowd']})
        except:
            m=stock

    d_kd_k   = pd.DataFrame(d_k,columns=stock_list)
    d_kd_d   = pd.DataFrame(d_d,columns=stock_list)        
    w_kd_k   = pd.DataFrame(w_k,columns=stock_list)
    w_kd_d   = pd.DataFrame(w_d,columns=stock_list)        
    d_ma20 = pd.DataFrame(ma_20,columns=stock_list) 

    c1 = w_kd_k.iloc[-1] > 60
    c2 = w_kd_d.iloc[-1] > 60
    c3 = w_kd_k.iloc[-1] < w_kd_d.iloc[-1]
    c4 = d_ma20.iloc[-1] > price.iloc[-1]
    c5 = 成交股數.iloc[-1] > 1000000
    c6 = ((price.iloc[-2]-price.iloc[-1])/price.iloc[-2]) > 0.02
    select_stock = c1 & c2 & c3 & c4 & c5 & c6
    
    # remove 28** 金融股
    for i in select_stock[select_stock].index:
        try: 
            if(int(i) > 2800) :
                if(int(i) < 2899) :
                    select_stock[i]=0
                    print("移除金融股:",i)
        except:
            m=i
    #print(select_stock[select_stock])
    #select_stock = c1 & c2 & c3 & c4
    return select_stock[select_stock]